# Travail préparatoire

Avant de commencer le TP, nous vous recommandons fortement de :
- Suivre le tutoriel [Customize what happens in Model.fit](https://www.tensorflow.org/guide/keras/customizing_what_happens_in_fit)
- Vous reporter si besoin à la documentation de la classe [``` Model ```](https://www.tensorflow.org/api_docs/python/tf/keras/Model)

# Objectifs 

Les objectifs de ce TP sont : 
- Découvrir le principe de la compression des réseaux de neurones par distillation (cf: Hinton et al., [Distilling the Knowledge in a Neural Network](https://arxiv.org/abs/1503.02531))

- Illustrer la flexibilité du paradigme des réseaux de neurones profonds

- Apprendre à utiliser la classe ``` Model ``` et à redéfinir certaines de ses méthodes (cf. travail préparatoire)





# Travail à réaliser 

Lors de ce TP, nous allons réaliser la distillation d'un réseau de neurones de type CNN (le *Teacher*) dans un réseau plus léger (le "Student"). Pour ce cas d'étude nous utiliserons la base MNIST que nous avons déjà utilisé lors des TP précédents. 
Vous allez donc devoir : 
- Définir l'architecture du réseau *Teacher* et optimiser le modèle sur la base MNIST (à noter que vous pouvez également utiliser un modèle pré-appris pour la tâche qui nous intéresse, ici la reconnaissance de chiffre manuscript)
- Définir l'architecture du réseau léger *Student*
- Préparer les données d'apprentissage qui serviront à la distillation. Nous utiliserons les données de MNIST (les mêmes que celles qui ont servi à l'apprentissage du Teacher, mais ce n'est pas une obligation, les deux bases peuvent être différentes, seules les tâches à réaliser doivent être identiques)
- Implémenter la classe *Distiller* qui sera en charge de la distillation. 

La classe ``` Distiller ``` héritera de la classe ``` Model ``` pour laquelle il faudra redéfinir le constructeur, et les méthodes ``` train_step ``` et ``` test_step ```. Vous pourrez également redéfinir la méthode ``` compile ``` si vous souhaitez faire un code plus générique et tester différentes fonctions de coût et hyper-paramètre propre à la méthode de distillation.





In [ ]:
import tensorflow as tf
from tensorflow.keras import Model

## Préparation des données 

In [ ]:
## Chargement et normalisation des données
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images / 255.0
test_images = test_images / 255.0

# POUR LES CNN : On rajoute une dimension pour spécifier qu'il s'agit d'images en NdG
train_images = train_images.reshape(-1,28,28,1)
test_images = test_images.reshape(-1,28,28,1)

# One hot encoding
train_labels = tf.keras.utils.to_categorical(train_labels)
test_labels = tf.keras.utils.to_categorical(test_labels)

## Définition et apprentissage de modèle ```teacher```


Définition du modèle

In [ ]:
## DEFINITION DES MODELES
## Teacher 
## Définition de l'architecture du modèle
teacher = tf.keras.models.Sequential()
teacher.add(tf.keras.layers.Conv2D(filters=16,kernel_size=(3,3),padding="same", activation='relu', input_shape=(28, 28, 1)))
teacher.add(tf.keras.layers.AveragePooling2D())
teacher.add(tf.keras.layers.Conv2D(filters=32,kernel_size=(3,3),padding="same", activation='relu'))
teacher.add(tf.keras.layers.AveragePooling2D())
teacher.add(tf.keras.layers.Conv2D(filters=64,kernel_size=(3,3),padding="same", activation='relu'))
teacher.add(tf.keras.layers.AveragePooling2D())
teacher.add(tf.keras.layers.Flatten())
teacher.add(tf.keras.layers.Dense(1024 , activation='relu'))
teacher.add(tf.keras.layers.Dense(512 , activation='relu'))
teacher.add(tf.keras.layers.Dense(10))
print(teacher.summary())


Apprentissage du modèle (Adam + Entropie Croisée sur 10 epochs)


In [ ]:
load_teacher = False

if (load_teacher == True):
  teacher = tf.keras.models.load_model('saved_teacher_with_T')
else:
  teacher.compile(
      optimizer=tf.keras.optimizers.Adam(), 
      loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), 
      metrics=tf.keras.metrics.CategoricalAccuracy()
    )

  teacher.fit(train_images, train_labels,batch_size=64,epochs=20)
  tf.keras.models.save_model(
      teacher, 'saved_teacher_with_T', overwrite=True, include_optimizer=True
    )

Evaluation des performances sur la base de test

In [ ]:
teacher.evaluate(test_images, test_labels)

## Définition du modèle  ```student```


In [ ]:
## Student
student = tf.keras.models.Sequential()
# Expliquez à quoi correspondent les valeurs numériques qui définissent les couches du réseau
student.add(tf.keras.layers.Conv2D(filters=8,kernel_size=(3,3),padding="same", activation='relu', input_shape=(28, 28, 1)))
student.add(tf.keras.layers.AveragePooling2D())
student.add(tf.keras.layers.Conv2D(filters=8,kernel_size=(3,3),padding="same", activation='relu'))
student.add(tf.keras.layers.AveragePooling2D())
student.add(tf.keras.layers.Flatten())
student.add(tf.keras.layers.Dense(64 , activation='relu'))
student.add(tf.keras.layers.Dense(32 , activation='relu'))
student.add(tf.keras.layers.Dense(10))
# expliquer le nombre de paramètre de ce réseau
print(student.summary())

In [ ]:
# On copie l'instance 
student_loss_sup =  tf.keras.models.clone_model(student)
student_loss_distillation =  tf.keras.models.clone_model(student)
student_loss_both =  tf.keras.models.clone_model(student)


##Définition de la classe ``` Distiller ```

Le distiller a besoin du modèle ``` teacher ``` appris et de modèle ``` student ``` 

Les méthodes ``` train_step ``` et ``` test_step ``` doit être redéfinies et seront appelées respectivement par les méthodes ``` fit ``` et  ``` evaluate ```



In [ ]:
class Distiller(Model):

    def __init__(self, teacher, student, alpha, T):
        super(Distiller, self).__init__()
        self.teacher = teacher
        self.student = student
        self.alpha = alpha
        self.T = T
    
    def train_step(self, data):
        # on récupère les données
        img_batch, label_batch = data

        # Prediction du teacher (pour guider l'apprentissage du student)
        pred_teacher = self.teacher(img_batch)
        
        # Prédition du student
        with tf.GradientTape() as tape: 
            pred_student = self.student(img_batch)
            loss_distillation = tf.keras.losses.categorical_crossentropy(
                tf.nn.softmax(pred_teacher / self.T, axis=1),
                tf.nn.softmax(pred_student / self.T, axis=1),               
               from_logits = True
              )
            loss_sup = tf.keras.losses.categorical_crossentropy(
                label_batch,
                pred_student,
                from_logits = True
              )
            loss = self.alpha*loss_distillation + (1-self.alpha)*loss_sup



        # Calcul des gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Mise à jour des poids
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`. Necessaire ? pas dans compile)
        self.compiled_metrics.update_state(label_batch, pred_student)

        # Retourne un dictionnaire avec le résultats
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {
                "loss_distillation": loss_distillation, 
                "loss_sup": loss_sup, 
                "loss": loss
            }
        )
        return results

    def test_step(self, data):
        # on récupère les données
        img_batch, label_batch = data
        # Compute predictions
        pred_student = self.student(img_batch, training=False)

        # Calculate the loss
        student_loss = tf.keras.losses.categorical_crossentropy(label_batch, pred_student, from_logits=True)

        # Update the metrics.
        self.compiled_metrics.update_state(label_batch, pred_student)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results

## Distillation du modèle 

Apprentissage du modèle léger

In [ ]:
# Uniquement la loss superviée
distiller = Distiller(student=student_loss_sup, teacher=teacher, alpha=1, T=1)
distiller.compile(optimizer=tf.keras.optimizers.Adam(), metrics=[tf.keras.metrics.CategoricalAccuracy(name='précision :')])
distiller.fit(train_images, train_labels, epochs=5)

Evaluation du modèle

In [ ]:
distiller.evaluate(test_images, test_labels, batch_size=16)

In [ ]:
# Uniquement la loss distillation
distiller = Distiller(student=student_loss_sup, teacher=teacher, alpha=1, T=1)
distiller.compile(optimizer=tf.keras.optimizers.Adam(), metrics=[tf.keras.metrics.CategoricalAccuracy(name='précision :')])
distiller.fit(train_images, train_labels, epochs=5)
distiller.evaluate(test_images, test_labels, batch_size=16)

In [ ]:
# les 2 loss
distiller = Distiller(student=student_loss_both, teacher=teacher, alpha=0.5, T=1)
distiller.compile(optimizer=tf.keras.optimizers.Adam(), metrics=[tf.keras.metrics.CategoricalAccuracy(name='précision :')])
distiller.fit(train_images, train_labels, epochs=5)
distiller.evaluate(test_images, test_labels, batch_size=16)

Dans ce TP, nous avons implémenté et évaluer une stratégie de distillation de l'information d'un réseau Teacher (expert) vers un réseau Student La distillation peut-être utilisée pour :
- compresser la taille (nombre de paramètre) d'un réseau expert
- spécialiser un réseau léger pour un domaine particulier
- apprendre un réseau lorsque l'on dispose d'un (ou plusieurs) réseau mais pas de données annotées. 


Vous pouvez également tester cette stratégie sur : 
- d'autres bases (e.g. CIFAR 10)
- en utilisant des réseaux pré-apris disponibles dans TF2 (eg: https://tfhub.dev/deepmind/ganeval-cifar10-convnet/1) -> cf: https://www.tensorflow.org/tutorials/images/transfer_learning_with_hub pour un exemple d'utilisation de modèles pré-appris